In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Install required packages
!pip install wandb torch torchvision pandas numpy matplotlib seaborn

# Set up Kaggle API
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [15]:
# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [16]:
# Download the dataset
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip

 91% 259M/285M [00:00<00:00, 469MB/s]
100% 285M/285M [00:00<00:00, 476MB/s]


# Download artifacts

In [17]:
import wandb
import torch
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Login to wandb
wandb.login()

# Init run
run = wandb.init(project="facial-expression-recognition")

# Download artifacts
cnn_artifact = run.use_artifact('ellekvirikashvili-free-university-of-tbilisi-/facial-expression-recognition/run-30aso492-history:v0')
cnn_dir = cnn_artifact.download()

vit_artifact = run.use_artifact('ellekvirikashvili-free-university-of-tbilisi-/facial-expression-recognition/vit-fer2013-final-model:v0')
vit_dir = vit_artifact.download()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [18]:
class TestDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixels = self.data.iloc[idx]['pixels']
        image = np.array(pixels.split(), dtype='uint8')
        image = image.reshape(48, 48, 1).astype('float32') / 255.0
        image = np.repeat(image, 3, axis=-1)

        if self.transform:
            image = self.transform(image)

        return image


In [19]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),  # for ViT — you can skip or change for CNN
])

test_dataset = TestDataset(test_df, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [2]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ellekvirikashvili-free-university-of-tbilisi-/facial-expression-recognition/vit-fer2013-final-model:v0', type='model')
artifact_dir = artifact.download()

wandb: Currently logged in as: ellekvirikashvili (ellekvirikashvili-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


# Download model from wandb

In [3]:
import torch
from timm import create_model

# Load the full checkpoint dictionary
checkpoint = torch.load(f"{artifact_dir}/final_vit_model.pth")

# Recreate the architecture
model = create_model('mobilevit_xxs', pretrained=False, num_classes=7)

# Load the weights from the 'model_state_dict' key
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("Checkpoint loaded successfully ✅")


RuntimeError: Error(s) in loading state_dict for ByobNet:
	Missing key(s) in state_dict: "stem.conv.weight", "stem.bn.weight", "stem.bn.bias", "stem.bn.running_mean", "stem.bn.running_var", "stages.0.0.conv1_1x1.conv.weight", "stages.0.0.conv1_1x1.bn.weight", "stages.0.0.conv1_1x1.bn.bias", "stages.0.0.conv1_1x1.bn.running_mean", "stages.0.0.conv1_1x1.bn.running_var", "stages.0.0.conv2_kxk.conv.weight", "stages.0.0.conv2_kxk.bn.weight", "stages.0.0.conv2_kxk.bn.bias", "stages.0.0.conv2_kxk.bn.running_mean", "stages.0.0.conv2_kxk.bn.running_var", "stages.0.0.conv3_1x1.conv.weight", "stages.0.0.conv3_1x1.bn.weight", "stages.0.0.conv3_1x1.bn.bias", "stages.0.0.conv3_1x1.bn.running_mean", "stages.0.0.conv3_1x1.bn.running_var", "stages.1.0.conv1_1x1.conv.weight", "stages.1.0.conv1_1x1.bn.weight", "stages.1.0.conv1_1x1.bn.bias", "stages.1.0.conv1_1x1.bn.running_mean", "stages.1.0.conv1_1x1.bn.running_var", "stages.1.0.conv2_kxk.conv.weight", "stages.1.0.conv2_kxk.bn.weight", "stages.1.0.conv

# Display Model Architecture

In [7]:
print(model)


ByobNet(
  (stem): ConvNormAct(
    (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNormAct2d(
      16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
  )
  (stages): Sequential(
    (0): Sequential(
      (0): BottleneckBlock(
        (shortcut): Identity()
        (conv1_1x1): ConvNormAct(
          (conv): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
        )
        (conv2_kxk): ConvNormAct(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
          

In [4]:
checkpoint = torch.load(f"{artifact_dir}/final_vit_model.pth")
print(checkpoint.keys())


dict_keys(['model_state_dict', 'optimizer_state_dict', 'test_accuracy', 'config'])


In [5]:
state_dict = checkpoint['model_state_dict']


In [6]:
state_dict = checkpoint['model_state_dict']
for key in list(state_dict.keys())[:10]:  # just first 10 keys to peek
    print(key)


model.stem.conv.weight
model.stem.bn.weight
model.stem.bn.bias
model.stem.bn.running_mean
model.stem.bn.running_var
model.stem.bn.num_batches_tracked
model.stages.0.0.conv1_1x1.conv.weight
model.stages.0.0.conv1_1x1.bn.weight
model.stages.0.0.conv1_1x1.bn.bias
model.stages.0.0.conv1_1x1.bn.running_mean


In [7]:
from collections import OrderedDict

# Fix the keys
fixed_state_dict = OrderedDict()
for k, v in state_dict.items():
    new_k = k.replace("model.", "", 1)  # Remove only the first 'model.'
    fixed_state_dict[new_k] = v

# Load it
model.load_state_dict(fixed_state_dict)


<All keys matched successfully>

In [8]:
import pandas as pd

test_df = pd.read_csv('test.csv')

In [14]:
from torchvision import transforms
from PIL import Image


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [15]:
test_dataset = TestDataset(test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [16]:
import numpy as np

model.to(device)
model.eval()
predictions = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())


# Generate Submissions

In [23]:
submission_df = test_df.copy()
submission_df['emotion'] = predictions  # or vit_preds or your own blend
submission_df = submission_df[['emotion']]  # make sure the format matches what's required

submission_df.to_csv("submission.csv", index=False)


ეს არის მოდელები, რაც არ გავუშვი იდეაში, მაგრამ არტიფაქტად შენახული გვაქვს. ამ კოდში ვხედავთ როგორ შეგვიძლია ნებისმიერი მოდელიდან არქიტექტურის ამოღება.

In [ ]:
import wandb
import torch
import os
import glob
from pathlib import Path

def inspect_model_directory():
    """Inspect the downloaded models directory structure"""
    print("Inspecting models directory...")
    
    models_dir = Path("models")
    if not models_dir.exists():
        print("Models directory doesn't exist!")
        return
    
    # List all files and subdirectories
    for root, dirs, files in os.walk(models_dir):
        level = root.replace(str(models_dir), '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            file_path = os.path.join(root, file)
            file_size = os.path.getsize(file_path) / (1024*1024)  # MB
            print(f"{subindent}{file} ({file_size:.2f} MB)")

def inspect_model_structure(model_path):
    """Inspect the structure of a saved PyTorch model"""
    print(f"\nInspecting model: {model_path}")
    
    try:
        # Load the model state dict
        checkpoint = torch.load(model_path, map_location='cpu')
        
        # Check if it's a full checkpoint or just state_dict
        if isinstance(checkpoint, dict):
            if 'model_state_dict' in checkpoint:
                state_dict = checkpoint['model_state_dict']
                print("Found model_state_dict in checkpoint")
                if 'epoch' in checkpoint:
                    print(f"Epoch: {checkpoint['epoch']}")
                if 'best_acc' in checkpoint:
                    print(f"Best accuracy: {checkpoint['best_acc']}")
            elif 'state_dict' in checkpoint:
                state_dict = checkpoint['state_dict']
                print("Found state_dict in checkpoint")
            else:
                state_dict = checkpoint
                print("Checkpoint appears to be direct state_dict")
        else:
            state_dict = checkpoint
            print("Model appears to be direct state_dict")
        
        print(f"\nModel layers ({len(state_dict)} total):")
        for i, (key, tensor) in enumerate(state_dict.items()):
            if i < 20:  # Show first 20 layers
                print(f"  {key}: {tensor.shape}")
            elif i == 20:
                print(f"  ... and {len(state_dict) - 20} more layers")
                break
        
        return state_dict
        
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def download_and_inspect_wandb_models():
    """Download models from wandb and inspect their structure"""
    
    # Initialize wandb
    wandb.login()
    api = wandb.Api()
    
    # Model run information
    models_info = [
        {"run_id": "30aso492", "version": "attention_cnn_v2"},
        {"run_id": "j96barvg", "version": "attention_cnn_v3"}, 
        {"run_id": "940fei4e", "version": "attention_cnn_v4"},
        {"run_id": "4a37ougz", "version": "attention_cnn_v1"}
    ]
    
    model_files = {}
    
    print("Downloading and inspecting CNN models...")
    for model_info in models_info:
        try:
            run = api.run(f"ellekvirikashvili-free-university-of-tbilisi-/facial-expression-recognition/{model_info['run_id']}")
            
            # Create directory
            model_dir = f"models/{model_info['version']}"
            os.makedirs(model_dir, exist_ok=True)
            
            # Download files
            for file_name in ['best_attention_cnn.pth', 'final_attention_cnn.pth']:
                try:
                    file_path = run.file(file_name).download(root=model_dir, replace=True)
                    full_path = os.path.join(model_dir, file_name)
                    model_files[f"{model_info['version']}_{file_name.replace('.pth', '')}"] = full_path
                    print(f"Downloaded: {full_path}")
                    
                    # Inspect the model structure
                    inspect_model_structure(full_path)
                    
                except Exception as e:
                    print(f"Failed to download {file_name}: {e}")
                    
        except Exception as e:
            print(f"Failed to access run {model_info['run_id']}: {e}")
    
    # Download ViT model
    print("\nDownloading and inspecting ViT model...")
    try:
        artifact = api.artifact('ellekvirikashvili-free-university-of-tbilisi-/facial-expression-recognition/vit-fer2013-final-model:v0')
        vit_dir = "models/vit_model"
        os.makedirs(vit_dir, exist_ok=True)
        artifact.download(root=vit_dir, replace=True)
        
        # Find all files in ViT directory
        vit_files = glob.glob(f"{vit_dir}/**/*", recursive=True)
        for file_path in vit_files:
            if os.path.isfile(file_path):
                print(f"ViT file: {file_path}")
                if file_path.endswith('.pth') or file_path.endswith('.bin'):
                    inspect_model_structure(file_path)
                    
        model_files["vit_model"] = vit_dir
        
    except Exception as e:
        print(f"Failed to download ViT model: {e}")
    
    return model_files

def load_model_from_checkpoint(model_path):
    """Load model from checkpoint without defining architecture first"""
    print(f"\nLoading model from: {model_path}")
    
    try:
        checkpoint = torch.load(model_path, map_location='cpu')
        
        # Extract state_dict
        if isinstance(checkpoint, dict):
            if 'model_state_dict' in checkpoint:
                state_dict = checkpoint['model_state_dict']
            elif 'state_dict' in checkpoint:
                state_dict = checkpoint['state_dict']
            else:
                state_dict = checkpoint
        else:
            state_dict = checkpoint
        
        # Try to infer model structure from state_dict keys
        print("Model structure inferred from state_dict:")
        conv_blocks = []
        att_blocks = []
        fc_layers = []
        
        for key in state_dict.keys():
            if key.startswith('conv'):
                block_name = key.split('.')[0]
                if block_name not in conv_blocks:
                    conv_blocks.append(block_name)
            elif key.startswith('att'):
                block_name = key.split('.')[0]
                if block_name not in att_blocks:
                    att_blocks.append(block_name)
            elif key.startswith('fc'):
                fc_layers.append(key)
        
        print(f"Conv blocks: {conv_blocks}")
        print(f"Attention blocks: {att_blocks}")
        print(f"FC layers: {fc_layers[:10]}...")  # First 10 FC layer keys
        
        return state_dict, checkpoint
        
    except Exception as e:
        print(f"Error loading checkpoint: {e}")
        return None, None

def make_predictions_with_loaded_model(state_dict, test_data):
    """Make predictions using loaded state_dict (you'll need to reconstruct model)"""
    # This is where you'd need to reconstruct the exact model architecture
    # based on the state_dict structure we discovered above
    print("To make predictions, you need to reconstruct the model architecture")
    print("based on the state_dict structure shown above.")
    
    # Example of what you'd need to do:
    # model = YourActualModelClass()
    # model.load_state_dict(state_dict)
    # model.eval()
    # return model



In [ ]:

"""Main inspection function"""
print("="*60)
print("WANDB MODEL INSPECTOR")
print("="*60)

# First check if models directory already exists
inspect_model_directory()

# Download and inspect models
model_files = download_and_inspect_wandb_models()

print("\n" + "="*60)
print("SUMMARY OF DOWNLOADED MODELS")
print("="*60)

for model_name, path in model_files.items():
    print(f"{model_name}: {path}")
    if path.endswith('.pth'):
        state_dict, checkpoint = load_model_from_checkpoint(path)

print("\n" + "="*60)
print("NEXT STEPS")
print("="*60)
print("1. Use the model structure information above to recreate your model classes")
print("2. Load the state_dict into your model")
print("3. Run predictions on your test set")
print("\nExample:")
print("model = YourModelClass()")
print("model.load_state_dict(torch.load('path_to_model.pth'))")
print("model.eval()")

